In [6]:
!pip install transformers scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sentiment_analysis import Sentiment_Analysis
import pandas as pd

# Import Drive API and authenticate.
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
df_mercedes = pd.read_csv('mercedes_v2.csv')
model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'

sa_model = Sentiment_Analysis(model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
mercedes_clean_tweets = sa_model.preprocess_tweets(df_mercedes)
rated_tweets_mercedes = sa_model.sentiment_analysis(mercedes_clean_tweets)

df_mercedes[['sentiment', 'sentiment_confidence']] = rated_tweets_mercedes
df_mercedes.head()

,text,id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user,tagged_ppl,num_tagged_ppl,hashtags,num_hashtags,sentiment,sentiment_confidence
0,Mr @MercedesAMG. 🔥 https://t.co/O91guhESXI,1567235154690580484,2022-09-06T19:35:53.000Z,503,142,10754,80,MercedesAMGF1,@MercedesAMG.,1.0,NaN,NaN,Positive,0.945784
1,Swapping the 🧡 of Zandvoort for the ❤️ of Monz...,1567221701036867587,2022-09-06T18:42:25.000Z,111,43,3197,2,MercedesAMGF1,@F1,1.0,NaN,NaN,Positive,0.59351
2,Another one. 🏆 https://t.co/Ru3LcHbWwf,1567209084885127170,2022-09-06T17:52:17.000Z,111,59,4164,23,MercedesAMGF1,NaN,NaN,NaN,NaN,Positive,0.641124
3,Monza. Simply magic. 😍 Our #ItalianGP NFT x @...,1567205534549377024,2022-09-06T17:38:11.000Z,34,37,660,6,MercedesAMGF1,@FTX_Official,1.0,#ItalianGP,1.0,Positive,0.97684
4,⭐ BOY. https://t.co/6sd6LuYRPQ,1567182788209557509,2022-09-06T16:07:48.000Z,507,210,13656,103,MercedesAMGF1,NaN,NaN,NaN,NaN,Neutral,0.859821


In [6]:
df_redbull = pd.read_csv('redbull_v2.csv')
redbull_clean_tweets = sa_model.preprocess_tweets(df_redbull)
rated_tweets_redbull = sa_model.sentiment_analysis(redbull_clean_tweets)

df_redbull[['sentiment', 'sentiment_confidence']] = rated_tweets_redbull
df_redbull.head()

,text,id,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,user,tagged_ppl,num_tagged_ppl,hashtags,num_hashtags,sentiment,sentiment_confidence
0,Racing on water 👊 When two worlds collide 👀 @O...,1567195567553404931,2022-09-06T16:58:35.000Z,89,13,1527,9,redbullracing,@Oracle @SailGP,2.0,NaN,NaN,Neutral,0.633909
1,Different kind of training for the Bulls today...,1567155092930928640,2022-09-06T14:17:45.000Z,252,62,6029,64,redbullracing,@Oracle @SailGP,2.0,NaN,NaN,Neutral,0.916004
2,Seeing double 👀 #DutchGP 🇳🇱 https://t.co/GNwcx...,1567143008637706245,2022-09-06T13:29:44.000Z,398,31,7668,15,redbullracing,NaN,NaN,#DutchGP,1.0,Neutral,0.679432
3,"Game, Seb and match 😉 Serving up Audio Moments...",1567096713449508864,2022-09-06T10:25:46.000Z,169,21,2966,13,redbullracing,NaN,NaN,#DutchGP,1.0,Neutral,0.615546
4,"An education in aero, behind the scenes in Zan...",1567072330844626944,2022-09-06T08:48:53.000Z,53,4,877,0,redbullracing,@Oracle,1.0,NaN,NaN,Neutral,0.637419


In [7]:
# Write the DataFrame to CSV file.
with open('/gdrive/My Drive/f1 data/redbull.csv', 'w') as f:
  df_redbull.to_csv(f)

# Making the dataset with the races



In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [5]:
data = {'GP':['Australian GP', 'Emilia Romagna GP', 'Miami GP', 'Spanish GP', 'Monaco GP', 'Azerbaijan GP', 'Canadian GP', 'British GP', 'Austrian GP', 'French GP', 'Hungarian GP', 'Belgian GP', 'Dutch GP'], 
        'Date':[dt.datetime(2022, 4, 10), dt.datetime(2022, 4, 24), dt.datetime(2022, 5, 8), dt.datetime(2022, 5, 22), dt.datetime(2022, 5, 29), dt.datetime(2022, 6, 12), dt.datetime(2022, 6, 19), dt.datetime(2022, 7, 3), dt.datetime(2022, 7, 10), dt.datetime(2022, 7, 24), dt.datetime(2022, 7, 31), dt.datetime(2022, 8, 28), dt.datetime(2022, 9, 4)],
        'Leclerc_Position': ['P1', 'P6', 'P2', 'DNF', 'P4', 'DNF', 'P5', 'P4', 'P1', 'DNF', 'P6', 'P6', 'P3'],
        'Leclerc_Points': [26, 8, 18, 0, 12, 0, 10, 12, 25, 0, 8, 8, 15],
        'Sainz_Position': ['DNF', 'DNF', 'P3', 'P4', 'P2', 'DNF', 'P2', 'P1', 'DNF', 'P5', 'P4', 'P3', 'P8'],
        'Sainz_Points': [0, 0, 15, 12, 18, 0, 19, 25, 0, 11, 12, 15, 4],
        'Verstappen_Position': ['DNF', 'P1', 'P1', 'P1', 'P3', 'P1', 'P1', 'P7', 'P2', 'P1', 'P1', 'P1', 'P1'],
        'Verstappen_Points': [0, 26, 26, 25, 15, 25, 25, 6, 19, 25, 25, 26, 26],
        'Perez_Position': ['P2', 'P2', 'P4', 'P2', 'P1', 'P2', 'DNF', 'P2', 'DNF', 'P4', 'P5', 'P2', 'P5'],
        'Perez_Points': [18, 18, 12, 19, 25, 19, 0, 18, 0, 12, 10, 18, 10],
        'Hamilton_Position': ['P4', 'P13', 'P6', 'P5', 'P8', 'P4', 'P3', 'P3', 'P3', 'P2', 'P2', 'DNF', 'P4'],
        'Hamilton_Points': [12, 0, 8, 10, 4, 12, 15, 15, 15, 18, 19, 0, 12],
        'Russel_Position' : ['P3', 'P4', 'P5', 'P3', 'P5', 'P3', 'P4', 'DNF', 'P4', 'P3', 'P3', 'P4', 'P2'],
        'Russel_Points': [15, 12, 10, 15, 10, 15, 12, 0, 12, 15, 15, 12, 18]}

df_races = pd.DataFrame(data)
df_races
 

,GP,Date,Leclerc_Position,Leclerc_Points,Sainz_Position,Sainz_Points,Verstappen_Position,Verstappen_Points,Perez_Position,Perez_Points,Hamilton_Position,Hamilton_Points,Russel_Position,Russel_Points
0,Australian GP,2022-04-10,P1,26,DNF,0,DNF,0,P2,18,P4,12,P3,15
1,Emilia Romagna GP,2022-04-24,P6,8,DNF,0,P1,26,P2,18,P13,0,P4,12
2,Miami GP,2022-05-08,P2,18,P3,15,P1,26,P4,12,P6,8,P5,10
3,Spanish GP,2022-05-22,DNF,0,P4,12,P1,25,P2,19,P5,10,P3,15
4,Monaco GP,2022-05-29,P4,12,P2,18,P3,15,P1,25,P8,4,P5,10
5,Azerbaijan GP,2022-06-12,DNF,0,DNF,0,P1,25,P2,19,P4,12,P3,15
6,Canadian GP,2022-06-19,P5,10,P2,19,P1,25,DNF,0,P3,15,P4,12
7,British GP,2022-07-03,P4,12,P1,25,P7,6,P2,18,P3,15,DNF,0
8,Austrian GP,2022-07-10,P1,25,DNF,0,P2,19,DNF,0,P3,15,P4,12
9,French GP,2022-07-24,DNF,0,P5,11,P1,25,P4,12,P2,18,P3,15


In [6]:
data_drivers = {'Name': ['Charles Leclerc', 'Carlos Sainz Jr.', 'Max Verstappen', 'Sergio Perez', 'Lewis Hamilton', 'George Russell'],
                'Team': ['ScuderiaFerrari', 'ScuderiaFerrari', 'redbullracing', 'redbullracing', 'MercedesAMGF1', 'MercedesAMGF1']}

df_drivers = pd.DataFrame(data_drivers)
df_drivers.head()

,Name,Team
0,Charles Leclerc,ScuderiaFerrari
1,Carlos Sainz Jr.,ScuderiaFerrari
2,Max Verstappen,redbullracing
3,Sergio Perez,redbullracing
4,Lewis Hamilton,MercedesAMGF1


In [7]:
# Import Drive API and authenticate.
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

with open('/gdrive/My Drive/f1 data/races.csv', 'w') as f:
  df_races.to_csv(f)

with open('/gdrive/My Drive/f1 data/drivers.csv', 'w') as f:
  df_drivers.to_csv(f)

Mounted at /gdrive
